In [5]:
import redis

In [6]:
client = redis.Redis(host='localhost',port=6379,db=0)


# Test connection

In [7]:
client.ping()

True

# Set / Get

## Basic

In [8]:
name='deleteme'
value= 1

In [9]:
client.set(name,value)

True

In [10]:
client.get(name)

b'1'

## Expiration

In [ ]:
name='deleteme'
value= 1
expiration = 3
from time import sleep

### By TTL: seconds

In [16]:
client.set(name=name,value=value,ex=expiration)
for i in range(4):
    print(client.get(name))
    sleep(1)


b'1'
b'1'
b'1'
None


### By TTL: millis

In [19]:
client.set(name=name,value=value,px=expiration)
for i in range(4):
    print(client.get(name))
    sleep(.001)


b'1'
b'1'
b'1'
None


### By Timestamp: seconds

In [21]:
from datetime import datetime , timedelta
now = datetime.now

In [23]:
expiration = now() + timedelta(seconds=3)
client.set(name=name,value=value,exat=expiration)
for i in range(4):
    print(client.get(name))
    sleep(1)


b'1'
b'1'
b'1'
None


### By Timestamp: millis

In [24]:
expiration = now() + timedelta(milliseconds=3)
client.set(name=name,value=value,pxat=expiration)
for i in range(4):
    print(client.get(name))
    sleep(.001)


b'1'
b'1'
b'1'
None


### Set TTL > Update Value (do not modify ttl)

In [36]:
expiration = now() + timedelta(milliseconds=3)
name='deleteme'
client.set(name=name,value=value,pxat=expiration)
print(client.get(name))
client.set(name=name,value=value+1,keepttl=True)
for i in range(4):
    print(client.get(name))
    exp_info=client.getex(name=name)
    break
    print(f'expiration:{exp_info}')
    sleep(.001)


b'1'
b'2'


### Set TTL > Read and Update TTL
warning: seems not to be returning the value. returns b'2' (redis-py '4.1.4')

In [64]:
name='deleteme'
value=111
expiration=now() + timedelta(milliseconds=3)
client.set(name=name,value=value,pxat=expiration)
for i in range(4):
    expiration = now() + timedelta(milliseconds=1)
    response=client.getex(name=name,pxat=now() + timedelta(milliseconds=3))
    print(f'expiration:{exp_info}',client.get(name=name))
    sleep(.001)

sleep(1)
print(client.get(name=name))

response=client.getex(name=name,pxat=now() + timedelta(milliseconds=3))
print(f'expiration:{exp_info}',client.get(name=name))

expiration:b'2' b'111'
expiration:b'2' b'111'
expiration:b'2' b'111'
expiration:b'2' b'111'
None
expiration:b'2' None


#### Alternative


In [61]:
client.execute_command('SET',b'deleteme',b'123')
response=client.execute_command('GETEX',b'deleteme','EX',3)
print(response)
sleep(3)
response = client.execute_command('GET',b'deleteme')
print(response)

b'123'
None


In [62]:
client.execute_command('SET',b'deleteme',b'123')
response=client.execute_command('GETEX',b'deleteme','pxat',3)
print(response)
sleep(1)
response = client.execute_command('GET',b'deleteme')
print(response)

b'123'
None


### Remove expiration

In [65]:
value=1
name='deleteme'
expiration=3
client.set(name,value,ex=expiration)
print(client.ttl(name))
client.persist(name=name)
print(client.ttl(name))

3
-1


## 